In [6]:
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config

# Connecting to Wordpress Database

In [7]:
host = config.host
database = config.database
user = config.user
password = config.password

conn = mysql.connector.connect(host = host, database = database, user = user, password = password)

# Wordpress wp_jet_appointment table

#### SQL query set to appointments

In [8]:
appointments = "SELECT user_id, status, service, provider, user_email, date, slot, slot_end \
                FROM wp_jet_appointments \
                ORDER BY slot asc"

#### Reading sql query and saving to csv

In [9]:
ap_data = pd.read_sql_query(appointments, conn)
ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)

#### Loading Appointment Data from csv

In [10]:
ap = pd.read_csv("assets/data/wp_jet_appointments.csv")
ap

,user_id,status,service,provider,user_email,date,slot,slot_end
0,NaN,pending,404,415,abrahammrjones@gmail.com,1591833600,1591876800,1591880400
1,10.0,pending,456,417,abrahammrjones@gmail.com,1591833600,1591880400,1591891200
2,10.0,pending,460,415,abrahammrjones@gmail.com,1591833600,1591880400,1591884000
3,10.0,pending,458,415,abrahammrjones@gmail.com,1591833600,1591884000,1591891200
4,10.0,pending,448,415,abrahammrjones@gmail.com,1591833600,1591884000,1591887600
5,10.0,pending,442,415,abrahammrjones@gmail.com,1591833600,1591902000,1591905600


#### Calculating duration times

In [11]:
duration = pd.to_datetime((ap.slot_end - ap.slot), unit='s').dt.time

#### Adding duration column to appointment dataframe

In [12]:
ap["duration"] = duration
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,415,abrahammrjones@gmail.com,1591833600,1591876800,1591880400,01:00:00
1,10.0,pending,456,417,abrahammrjones@gmail.com,1591833600,1591880400,1591891200,03:00:00
2,10.0,pending,460,415,abrahammrjones@gmail.com,1591833600,1591880400,1591884000,01:00:00
3,10.0,pending,458,415,abrahammrjones@gmail.com,1591833600,1591884000,1591891200,02:00:00
4,10.0,pending,448,415,abrahammrjones@gmail.com,1591833600,1591884000,1591887600,01:00:00
5,10.0,pending,442,415,abrahammrjones@gmail.com,1591833600,1591902000,1591905600,01:00:00
